In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Import library

In [ ]:
# basic modules 
import geopandas as gpd 
import pandas as pd 
import json 
import glob 
import os 

# visualizaztion modules 
from bokeh.resources import INLINE 
from bokeh.io import output_notebook, show, output_file 
from bokeh.plotting import figure 
from bokeh.models import GeoJSONDataSource, LinearColorMapper, ColorBar, CustomJS, ColumnDataSource
from bokeh.palettes import brewer,mpl 

# interactive visualization models 
from bokeh.io import curdoc, output_notebook 
from bokeh.models import Slider, HoverTool 
from bokeh.layouts import widgetbox, row, column

output_notebook()

# World Map Happiness Score data

In [ ]:
df = pd.read_csv('../input/world-happiness/2015.csv')
df.head()

In [ ]:
shapefile = '../input/shapefiles/Interactive Geographic Maps/ne_110m_admin_0_countries/ne_110m_admin_0_countries.shp'

# Read shapefile using Geopandas
gdf = gpd.read_file(shapefile)[['ADMIN', 'ADM0_A3', 'geometry']]

# Rename columns.
gdf.columns = ['country', 'country_code', 'geometry']

In [ ]:
datafiles_1 = ['../input/world-happiness/2019.csv', '../input/world-happiness/2018.csv']
datafiles_2 = ['../input/world-happiness/2015.csv', '../input/world-happiness/2016.csv']

df = pd.concat((pd.read_csv(f).assign(Year=os.path.basename(f[0:4])) for f in datafiles_1), sort=False)
df2 = pd.concat((pd.read_csv(f).assign(Year=os.path.basename(f[0:4])) for f in datafiles_2), sort=False)
df3 = pd.read_csv('../input/world-happiness/2017.csv').assign(Year="2017")

df['Country'] = df['Country or region']
df2['Score'] = df2['Happiness Score']
df3['Score'] = df3['Happiness.Score']

countries = pd.concat([df[['Country', 'Score', 'Year']], df2[['Country', 'Score', 'Year']], df3[['Country', 'Score', 'Year']]], sort=False)

countries = countries.replace('United States', 'United States of America')

In [ ]:
# 1. Define function that returns json_data for the year selected by slider.
def json_data(selectedYear):
    yr = selectedYear
    df_yr = countries[countries['Year'] == yr]
    merged = gdf.merge(df_yr, left_on ='country', right_on ='Country', how ='left')
    merged['Score'] = merged.Score.fillna("N/A")
    merged_json = json.loads(merged.to_json())
    json_data = json.dumps(merged_json)
    return json_data

# 2. Define the callback function.
# This is what will be called when the slider value changes.
def update_plot(attr, old, new):
    yr = slider.value
    new_data = json_data(str(yr))
    geosource.geojson = new_data
    p.title.text = 'Happiness Score by Country (%d)' %yr

In [ ]:
# Input GeoJSON source that contains features for plotting.
# Select a default year to show on first display.
geosource = GeoJSONDataSource(geojson = json_data("2016"))

# Define a sequential multi-hue color palette.
# I chose the Plasma theme because 10 would map to yellow :)
palette = mpl['Plasma'][10]

# Instantiate LinearColorMapper that maps numbers to a sequence of colors.
color_mapper = LinearColorMapper(palette = palette, low = 0, high = 10)

# Add the hovering tooltips.
hover = HoverTool(tooltips = [ ('Country','@country'),('Score', '@Score')])

# Create the color bar. 
color_bar = ColorBar(color_mapper=color_mapper, label_standoff=8,width = 500, height = 20, border_line_color=None,location = (0,0), orientation = 'horizontal')

# Create the figure object.
p = figure(title = 'World Happiness by Country (2016)', plot_height = 600 , plot_width = 950, toolbar_location = None, tools = [hover])
# Remove the grid lines. 
p.xgrid.grid_line_color = None
p.ygrid.grid_line_color = None
# Increase the title font size.
p.title.text_font_size = '12pt'

# Add the patch renderer to the figure. 
# Notice that this is where you provide the Score column from the json_data and the color mapper.
p.patches('xs','ys', source = geosource,fill_color = {'field' :'Score', 'transform' : color_mapper}, line_color = 'white', line_width = 0.5, fill_alpha = 1)

# Create the custom JavaScript callback
update_plot = CustomJS(args=dict(p=p), code=
                       '''
    yr = slider.value
    new_data = json_data(str(yr))
    geosource.geojson = new_data
    p.title.text = 'Happiness Score by Country (%d)' %yr
    p3_test.change.emit();
''')    

# Place the color bar below the map.
p.add_layout(color_bar, 'below')
    
# Make a slider object.
slider = Slider(title = 'Year',start = 2015, end = 2019, step = 1, value = 2016)
slider.js_on_change('value', update_plot)

# Make a column layout of widgetbox (slider) and plot, and add it to the current document
layout = column(p,widgetbox(slider))
curdoc().add_root(layout)

# Show the figure.
output_notebook(INLINE)
show(layout)